In [1]:
import pandas as pd
import preprocessor as p
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [236]:
df = pd.read_csv(r'C:\Users\avbalaji2\Desktop\Final_Year_Project\backend\training data\YahooFinance_tweets.csv')

In [237]:
df.head()

,date,identifier,tweets
0,2022-01-28 07:57:43+00:00,c,"Robinhood quarterly revenue misses estimates, ..."
1,2022-01-28 07:36:22+00:00,c,Inflation: Seniors struggle with a new era of ...
2,2022-01-28 07:13:08+00:00,c,"Ackman boosts Netflix, but here's why one bear..."
3,2022-01-28 06:51:46+00:00,c,Bentley CEO details $3.4 billion ‘transformati...
4,2022-01-28 06:30:51+00:00,c,Why Tesla is building a humanoid robot https:/...


In [238]:
df=df.drop('identifier',axis=1)


In [239]:
from datetime import datetime
indexes_to_drop = []
date_list = []
list_of_words = ['TSLA','Tesla','Electric Vehicle','Elon','Elon Musk','EV']
for index, row in df.iterrows():
    f = 0
    for i in list_of_words:
        if i in row['tweets']:
            f = f+1        
    if f==0:
        indexes_to_drop.append(index)
    else:
        date_list.append(datetime.strptime(row['date'],'%Y-%m-%d %H:%M:%S%z').date())
df.drop(df.index[indexes_to_drop], inplace=True)
  

In [240]:
df = df.drop('date',axis=1)

In [241]:
df['date'] = date_list

In [242]:
sentiment_scores = []
for index, row in df.iterrows():
    pred_sentences = p.clean(row.tweets)
    tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
    tf_outputs = model(tf_batch)
    tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
    label = tf.argmax(tf_predictions, axis=1)
    if label==0:
        sentiment_scores.append(-(tf_predictions.numpy()[0][label]))
    else:
        sentiment_scores.append((tf_predictions.numpy()[0][label]))

C:\Users\avbalaji2\AppData\Local\Temp\ipykernel_1272\3003785167.py:9: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sentiment_scores.append(-(tf_predictions.numpy()[0][label]))
C:\Users\avbalaji2\AppData\Local\Temp\ipykernel_1272\3003785167.py:11: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sentiment_scores.append((tf_predictions.numpy()[0][label]))


In [243]:
df.head()

,tweets,date
4,Why Tesla is building a humanoid robot https:/...,2022-01-28
6,Tesla is disappointing investors because it do...,2022-01-28
8,Lucid stock falls after rival Tesla warns abou...,2022-01-28
12,Tesla stock slide 8% over supply-chain woes an...,2022-01-28
21,Bentley CEO: EV interest from luxury buyers do...,2022-01-27


In [244]:
df['yahoo_score'] = sentiment_scores

In [245]:
df

,tweets,date,yahoo_score
4,Why Tesla is building a humanoid robot https:/...,2022-01-28,-0.509248
6,Tesla is disappointing investors because it do...,2022-01-28,-0.522069
8,Lucid stock falls after rival Tesla warns abou...,2022-01-28,0.540681
12,Tesla stock slide 8% over supply-chain woes an...,2022-01-28,0.509675
21,Bentley CEO: EV interest from luxury buyers do...,2022-01-27,-0.511968
...,...,...,...
2912,$TM’s EV battery plant “will have a tremendous...,2021-12-07,-0.538205
2917,Fmr. Tesla Field Quality Mgr. Steven Henkes on...,2021-12-07,-0.519255
2953,Tesla replacing faulty Autopilot cameras in so...,2021-12-06,-0.523145
2954,EV maker Lucid receives an SEC subpoena on a $...,2021-12-06,0.507301


In [246]:
df = df.drop('tweets',axis=1)

In [247]:
df.to_csv(r'C:\Users\avbalaji2\Desktop\Final_Year_Project\backend\training data\YahooFinance_tweets.csv')